# Cvičení

## Kosatce

Vrať se k práci s data o kosatcích v souboru [IRIS.csv](IRIS.csv). Tentokrát zkus, jak dobře dokážou algoritmy pracovat se všemi třemi druhy kosatců. Využij algoritmy `KNeighborsClassifier` a `SVC` ke klasifikaci každého vzorku do jednoho ze tří druhů kosance.

Postup je stejný jako v lekci:

- Rozděl data na vstupní a výstupní proměnné. 
- Využij `GridSearchCV` k nalezení nejlepšího parametru pro `KNeighborsClassifier` (počet sousedů) a `SVC` (strategie)
- Dále vyzkoušej, jestli by nebylo zajímavé převést data do více dimenzí. Porovnej výsledek lineárního kernelu (`kernel="linear"`) s kernelem (`kernel="rbf"`). Můžeš vyzkoušet obě strategie, tj. budeš mít ve slovníku `params` dva klíče a každý z klíčů bude mít seznam dvou možných hodnot. Jaká dvojice parametrů má nejlepší hodnotu metriky `accuracy`?

In [11]:
import pandas
import matplotlib.pyplot as plt
import numpy

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

data = pandas.read_csv("IRIS.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [12]:
X = data.drop(columns=["species"])
y = data["species"]
scaler = StandardScaler()
X = scaler.fit_transform(X)
model_1 = KNeighborsClassifier()
params_1 = {"n_neighbors": range(1, 31, 2)}
clf_1 = GridSearchCV(model_1, params_1, scoring="accuracy")
clf_1.fit(X, y)
print(clf_1.best_params_)
print(clf_1.best_score_)

{'n_neighbors': 5}
0.96


In [13]:
model_2 = SVC()
params_2 = {"decision_function_shape": ["ovo", "ovr"], "kernel": ["linear", "rbf"]}
clf_2 = GridSearchCV(model_2, params_2, scoring="accuracy")
clf_2.fit(X, y)
print(clf_2.best_params_)
print(clf_2.best_score_)

{'decision_function_shape': 'ovo', 'kernel': 'linear'}
0.9666666666666668


## Bonus: Poruchy

Stáhni si data o poruchách ze souboru [predictive_maintenance.csv](predictive_maintenance.csv). Význam sloupců je následující:

- UID: jedinečný identifikátor v rozsahu 1 až 10000,
- ID produktu: skládá se z písmene L, M nebo H pro nízkou (50 % všech výrobků), střední (30 %) a vysokou (20 %) variantu kvality výrobku a sériového čísla specifického pro danou variantu,
- teplota vzduchu (K),
- teplota procesu (K),
- otáčky (ot/min),
- točivý moment (Nm),
- opotřebení nástroje (min),
- označení "selhání stroje" (pokud k němu došlo).

Tvým úkolem je vytvořit model, který bude predikovat poruchu stroje.

Proveď následující postup:

- Vyřaď z datasetu sloupce (jsou dva), které nemají pro analýzu význam.
- Podívej se, kolik typů poruch bylo objeveno. Vyřaď nejméně častou poruchu.
- Rozděl data na vstupní proměnné a výstupní proměnnou.
- Vyzkoušej algoritmy Support Vector Machine a K Nearest Neighbors ke klasifikaci poruchy stroje. Porovnej, který algoritmus dosáhl lepších výsledků.

In [14]:
data = pandas.read_csv("predictive_maintenance.csv")
data.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [15]:
data = data.drop(columns=["UDI", "Product ID", "Target"])
data.groupby("Failure Type").size()

(10000, 7)

In [16]:
y = data["Failure Type"]

columns_to_encode = ["Type"]
columns_to_scale  = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]

scaler = StandardScaler()
ohe    = OneHotEncoder(sparse_output=False)

scaled_columns  = scaler.fit_transform(data[columns_to_scale]) 
encoded_columns = ohe.fit_transform(data[columns_to_encode])
X = numpy.concatenate([scaled_columns, encoded_columns], axis=1)

10000

In [17]:
model_1 = SVC()
params_1 = {"kernel": ["linear", "rbf"], "decision_function_shape": ["ovo", "ovr"]}

clf_1 = GridSearchCV(model_1, params_1, scoring="accuracy")
clf_1.fit(X, y)

print(clf_1.best_params_)
print(round(clf_1.best_score_, 2))

{'decision_function_shape': 'ovo', 'kernel': 'rbf'}
0.96


In [18]:
model_1 = KNeighborsClassifier()
params_1 = {"n_neighbors": [3, 5, 11, 15]}

clf_1 = GridSearchCV(model_1, params_1, scoring="accuracy")
clf_1.fit(X, y)

print(clf_1.best_params_)
print(round(clf_1.best_score_, 2))

{'n_neighbors': 15}
0.96
